In [1]:
# Dataset Location
raw_data_path = "raw_data"
cleaned_data_path= "cleaned_data"

In [3]:
import pandas as pd
# Inital Raw Data Used.

def clean_keggle_df(data_path):
    df = pd.read_csv(data_path)
    df = df[["questionText", "topics", "re_diagnosis","clean_answer_text"]]
    # Lower case
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    # remove non-world
    df = df.replace(to_replace=r'[^\w\s]', value="", regex=True)
    # remove number
    df = df.replace(to_replace=r'\d', value='', regex=True)

    return df

def clean_hugging_df(data_path):
    df = pd.read_csv(data_path)

    df = df[["questionTitle", "questionText", "topic", "answerText"]]
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    df["questionText"] = df["questionTitle"].fillna('') + " " + df["questionText"].fillna('')
    df = df.replace(to_replace=r'[^\w\s]', value="", regex=True)
    df = df.replace(to_replace=r'\d', value='', regex=True)
    df = df[["questionText", "topic", "answerText"]]

    # print(hugging_df.head)
    return df

keggle_df = clean_keggle_df(f"{raw_data_path}/counsel_cleaned.csv")   
print(keggle_df.shape)
keggle_df.to_csv(f"{cleaned_data_path}/cleaned_counsel.csv")
hugging_df = clean_hugging_df(f"{raw_data_path}/huggin_counsel_chat.csv")
print(hugging_df.shape)
hugging_df

(1373, 4)


/var/folders/hy/344rq3v160s1tb_mmkdw_fqc0000gn/T/ipykernel_87949/1977113534.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)


OSError: Cannot save file into a non-existent directory: 'cleaned_data'

In [4]:
# Chat Promt, Design.
combined_dataset = pd.read_csv(f"{cleaned_data_path}/combined_output.csv")
print(combined_dataset.columns)


Index(['Unnamed: 0', 'question_text', 'topics', 'answer_text'], dtype='object')


In [5]:
# Cleaned Combined Datasets too shorts and too long
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
text = "This is an example. Here is another sentence."
sentences = sent_tokenize(text)

print(sentences)


def is_noisy(text: str) -> bool:
    if re.search(r'[가-힣A-Za-z]', text) is None:
        return True
    cleaned = re.sub(r'[^\w\s]', '', text) 
    if len(cleaned) == 0 or len(cleaned) < len(text) * 0.02:  
        return True
    return False

def clean_combined_dataset(df):
    def token_count(text):
        tokens = sent_tokenize(text)
        return len(tokens)
    # Apply the token_count function to calculate the number of tokens in questions and answers
    df = df.dropna()
    df['q_token_count'] = df['question_text'].apply(token_count)
    df['a_token_count'] = df['answer_text'].apply(token_count)
    
    return df
# print(combined_dataset.columns)
print(f"Before: {combined_dataset.shape}")
cleaned_combined_df = clean_combined_dataset(combined_dataset)
cleaned_combined_df.to_csv(f"{cleaned_data_path}/cleaned_combined.csv")
print(f"After: {cleaned_combined_df.shape}")


['This is an example.', 'Here is another sentence.']
Before: (4148, 4)
After: (3985, 6)


[nltk_data] Downloading package punkt_tab to /Users/yoon/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
/var/folders/hy/344rq3v160s1tb_mmkdw_fqc0000gn/T/ipykernel_87949/1731113301.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['q_token_count'] = df['question_text'].apply(token_count)
/var/folders/hy/344rq3v160s1tb_mmkdw_fqc0000gn/T/ipykernel_87949/1731113301.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['a_token_count'] = df['answer_text'].apply(token_count)


In [9]:
# Lamma testing Device
from datasets import Dataset, DatasetDict

target_df = pd.read_csv(f"{cleaned_data_path}/cleaned_combined.csv")
print(f"After: {target_df.shape}")
print(f"Columns: {target_df.columns}")

system_prompt = (
    "You are a mental health assistant trained in cognitive behavioral therapy. "
    "Help users explore their thoughts and feelings with empathy and guidance."
)

def format_to_chat_messages(example):
    return {
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": example["question_text"] + f" based on {example['topics']}"},
            {"role": "assistant", "content": example["answer_text"]},
        ]
    }

# Grab only nessary columns only
target_df = target_df[["question_text", "topics", "answer_text"]].dropna()
hf_dataset = Dataset.from_pandas(target_df)

# Hugging Face Datasets
hf_dataset = hf_dataset.map(format_to_chat_messages)
hf_dataset = hf_dataset.remove_columns([col for col in hf_dataset.column_names if col != "messages"])



After: (3985, 7)
Columns: Index(['Unnamed: 0.1', 'Unnamed: 0', 'question_text', 'topics', 'answer_text',
       'q_token_count', 'a_token_count'],
      dtype='object')


Map: 100%|██████████| 3985/3985 [00:00<00:00, 54418.28 examples/s]


In [10]:
import json

output_path = f"{train_data_path}/train_dataset.jsonl"

with open(output_path, "w", encoding="utf-8") as f:
    for example in hf_dataset:
        json.dump({"messages": example["messages"]}, f, ensure_ascii=False)
        f.write("\n")

print(f"✅ Saved: {output_path}")


✅ Saved: train_data/train_dataset.jsonl
